In [1]:
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import requests
import scrapy
from PIL import Image
from bs4 import BeautifulSoup
from docx import Document
from scrapy.crawler import CrawlerProcess
from scrapy.crawler import CrawlerRunner
from scrapy.selector import Selector
from twisted.internet import reactor
from scrapy.utils.project import get_project_settings
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

# EJERCICIO 1

+ Scraping con Beautiful Soup y Selenium.

## PRIMERA WEB 
## "http://quotes.toscrape.com"
+ **BEAUTIFUL SOUP:**

In [2]:
URL = "http://quotes.toscrape.com"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

print(page.text)

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/login">Login</a>
                
                </p>
            </div>
        </div>
    

<div class="row">
    <div class="col-md-8">

    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">
        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
        <span>by <small class="author" itempr

Primero cargamos el código **HTML** de la página web usando la biblioteca **requests** de **bs4**. Podemos buscar las etiquetas y los aributos que representan los elementos de la página web, y asi extraerlos. Esta web consta de un total de 10 páginas consecutivas. Buscamos la cita, el autor, las etiquetas y la url en la que está alojada.

In [3]:
all_quotes = []
all_authors = []
all_tags = []
urls = []

def extract_information(soup):
    all_quotes = []
    all_authors = []
    all_tags = []

    quotes = soup.find_all('span', class_='text')
    authors = soup.find_all('small', class_='author')
    tag_containers = soup.find_all('div', class_='tags')

    for quote, author, tag_container in zip(quotes, authors, tag_containers):
        all_quotes.append(quote.text.strip())
        all_authors.append(author.text.strip())
        tags = [tag.text.strip() for tag in tag_container.find_all('a', class_='tag')]
        all_tags.append(tags)

    return all_quotes, all_authors, all_tags

base_url = 'http://quotes.toscrape.com'

for page_num in range(1, 11):
    url = f'{base_url}/page/{page_num}'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    quotes, authors, tags = extract_information(soup)
    urls.extend([url] * len(quotes))  
    all_quotes.extend(quotes)
    all_authors.extend(authors)
    all_tags.extend(tags)

for i, (quote, author, tags, url) in enumerate(zip(all_quotes, all_authors, all_tags, urls), start=1):
    print(f"--- Cita {i} ---")
    print("Cita:", quote)
    print("Autor:", author)
    print("Etiquetas:", tags)
    print("URL:", url)
    print()

--- Cita 1 ---
Cita: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Autor: Albert Einstein
Etiquetas: ['change', 'deep-thoughts', 'thinking', 'world']
URL: http://quotes.toscrape.com/page/1

--- Cita 2 ---
Cita: “It is our choices, Harry, that show what we truly are, far more than our abilities.”
Autor: J.K. Rowling
Etiquetas: ['abilities', 'choices']
URL: http://quotes.toscrape.com/page/1

--- Cita 3 ---
Cita: “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Autor: Albert Einstein
Etiquetas: ['inspirational', 'life', 'live', 'miracle', 'miracles']
URL: http://quotes.toscrape.com/page/1

--- Cita 4 ---
Cita: “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Autor: Jane Austen
Etiquetas: ['aliteracy', 'books', 'classic', 'humor']
URL: http://quotes.toscrape.com/page/1

--- Cita 5 ---

Creo listas para cada tipo de contenido, que utilizo a continuación para crear el dataframe **quotes_df**.

In [4]:
quotes_df = pd.DataFrame({'Quote': all_quotes, 'Author': all_authors, 'Tags': all_tags, 'URL': urls})

quotes_df

,Quote,Author,Tags,URL
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]",http://quotes.toscrape.com/page/1
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]",http://quotes.toscrape.com/page/1
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]",http://quotes.toscrape.com/page/1
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]",http://quotes.toscrape.com/page/1
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]",http://quotes.toscrape.com/page/1
...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy],http://quotes.toscrape.com/page/10
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,...",http://quotes.toscrape.com/page/10
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth],http://quotes.toscrape.com/page/10
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational],http://quotes.toscrape.com/page/10


In [5]:
quotes_df['Tags'] = [', '.join(tags) for tags in quotes_df['Tags']]

conteo_etiquetas = quotes_df['Tags'].str.split(', ', expand=True).stack().value_counts().head(10)

print(conteo_etiquetas)

love                    14
inspirational           13
life                    13
humor                   12
books                   11
reading                  7
friendship               5
truth                    4
friends                  4
attributed-no-source     3
dtype: int64


Para gestionar la columna **'Tags'**, mi primera idea fue convertirla en dummy, pero al hacerlo acabé teniendo un df con 140 columnas que era poco legible y manejable. 

Ante esta situación, decido realizar un conteo para ver cuáles son las etiquetas más repetidas y con esta información, creo una columna binaria para cada una de las cinco primeras. Estas columnas binarias indican la presencia o ausencia de cada etiqueta en una cita específica. Para manejar las etiquetas menos frecuentes, agrupé todas las demás en una única columna denominada **'Other_Tags'**. Esto simplifica el DataFrame al tiempo que conserva la información sobre las etiquetas menos comunes.

In [6]:
cinco_primeras_etiquetas = conteo_etiquetas.head(5).index.tolist()

for etiqueta in cinco_primeras_etiquetas:
    quotes_df[etiqueta] = quotes_df['Tags'].str.contains(etiqueta).astype(int)

def otras_etiquetas(row):
    etiquetas = row['Tags'].split(', ')
    otras = [etiqueta for etiqueta in etiquetas if etiqueta not in cinco_primeras_etiquetas]
    return ', '.join(otras) if otras else None

quotes_df['Other_Tags'] = quotes_df.apply(otras_etiquetas, axis=1)

quotes_df = quotes_df.drop(columns=['Tags'])

quotes_df

,Quote,Author,URL,love,inspirational,life,humor,books,Other_Tags
0,“The world as we have created it is a process ...,Albert Einstein,http://quotes.toscrape.com/page/1,0,0,0,0,0,"change, deep-thoughts, thinking, world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,http://quotes.toscrape.com/page/1,0,0,0,0,0,"abilities, choices"
2,“There are only two ways to live your life. On...,Albert Einstein,http://quotes.toscrape.com/page/1,0,1,1,0,0,"live, miracle, miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,http://quotes.toscrape.com/page/1,0,0,0,1,1,"aliteracy, classic"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,http://quotes.toscrape.com/page/1,0,1,0,0,0,be-yourself
...,...,...,...,...,...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,http://quotes.toscrape.com/page/10,0,0,1,0,0,better-life-empathy
96,“You have to write the book that wants to be w...,Madeleine L'Engle,http://quotes.toscrape.com/page/10,0,0,0,0,1,"children, difficult, grown-ups, write, writers..."
97,“Never tell the truth to people who are not wo...,Mark Twain,http://quotes.toscrape.com/page/10,0,0,0,0,0,truth
98,"“A person's a person, no matter how small.”",Dr. Seuss,http://quotes.toscrape.com/page/10,0,1,0,0,0,None


+ **SELENIUM:** 

En este caso, uso la biblioteca Selenium para el scraping.

In [7]:
driver = webdriver.Chrome()  

base_url = 'http://quotes.toscrape.com'

all_information = []

for page_num in range(1, 11):  
    url = f'{base_url}/page/{page_num}'
    driver.get(url)

    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'quote')))
    
    quotes = driver.find_elements(By.CLASS_NAME, 'quote')
    
    for quote in quotes:
        text = quote.find_element(By.CLASS_NAME, 'text').text.strip()
        author = quote.find_element(By.CLASS_NAME, 'author').text.strip()
        tags_elements = quote.find_elements(By.CLASS_NAME, 'tag')
        tags = [tag.text.strip() for tag in tags_elements]
        all_information.append((text, author, tags))

for i, (text, author, tags) in enumerate(all_information, start=1):
    print(f"--- Cita {i} ---")
    print("Cita:", text)
    print("Autor:", author)
    print("Etiquetas:", tags)
    print()

driver.quit()

--- Cita 1 ---
Cita: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Autor: Albert Einstein
Etiquetas: ['change', 'deep-thoughts', 'thinking', 'world']

--- Cita 2 ---
Cita: “It is our choices, Harry, that show what we truly are, far more than our abilities.”
Autor: J.K. Rowling
Etiquetas: ['abilities', 'choices']

--- Cita 3 ---
Cita: “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Autor: Albert Einstein
Etiquetas: ['inspirational', 'life', 'live', 'miracle', 'miracles']

--- Cita 4 ---
Cita: “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Autor: Jane Austen
Etiquetas: ['aliteracy', 'books', 'classic', 'humor']

--- Cita 5 ---
Cita: “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Autor: Marilyn Monroe
Etiquetas: ['be

Ahora probamos los mismos métodos en una web no tan sencilla y con más elementos.

## SEGUNDA WEB
## "https://es.wikipedia.org/wiki/David_Lynch"
+ **BEAUTIFUL SOUP:**

Empiezo igual que con la web anterior, cargando el codigo HTML de toda la página.

In [8]:
URL = "https://es.wikipedia.org/wiki/David_Lynch"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

print(soup.prettify())   

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   David Lynch - Wikipedia, la enciclopedia libre
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-

De este código extraemos el segundo nivel de encabezados. El primero (h1), era simplemente David Lynch.

In [9]:
encabezados_h2 = soup.find_all("h2")

for encabezado in encabezados_h2:
    print(encabezado.text.strip())

Contenidos
Biografía y obra[editar]
Influencias[editar]
Colaboradores habituales[editar]
Otras actividades[editar]
Vida personal[editar]
Filmografía[editar]
Discografía[editar]
Premios y distinciones[editar]
Referencias[editar]
Enlaces externos[editar]


En el siguiente paso, extraigo información de las tablas de la página web.

In [10]:
tablas = soup.find('table', {'class':'wikitable'})
saved_rowspans = []
inserted_cells = []

for row in tablas.findAll("tr"):
    cells = row.findAll(["th", "td"])
    print("Contenido de cells:", cells)  
    
    while len(saved_rowspans) < len(cells):
        saved_rowspans.append(None)

    for index, cell in enumerate(cells):
        if cell.has_attr("rowspan"):
            rowspan_data = {
                'rows_left': int(cell["rowspan"]),
                'value': cell,
            }
            saved_rowspans[index] = rowspan_data
            print("Rowspan found at index:", index, "with value:", cell.text)
        else:
            if len(cells) != len(saved_rowspans):
                for idx, rowspan_data in enumerate(saved_rowspans):
                    if rowspan_data is not None:
                        inserted_cells.append(rowspan_data['value'])
                        if rowspan_data['rows_left'] == 1:
                            saved_rowspans[idx] = None
                        else:
                            saved_rowspans[idx]['rows_left'] -= 1

    cells += inserted_cells
    inserted_cells = []  
    
    for cell in cells:
        print(cell.text.strip())  

Contenido de cells: [<th>Nombre
</th>, <th>Rol
</th>, <th>Ejemplos
</th>, <th>Cantidad de colaboraciones
</th>]
Nombre
Rol
Ejemplos
Cantidad de colaboraciones
Contenido de cells: [<td><a href="/wiki/Angelo_Badalamenti" title="Angelo Badalamenti">Angelo Badalamenti</a>
</td>, <td>Compositor
</td>, <td><i>Blue Velvet, Twin Peaks, Wild at Heart, Lost Highway</i> y <i>Mulholland Drive</i>.
</td>, <td>12 producciones
</td>]
Angelo Badalamenti
Compositor
Blue Velvet, Twin Peaks, Wild at Heart, Lost Highway y Mulholland Drive.
12 producciones
Contenido de cells: [<td><a href="/wiki/Mary_Sweeney" title="Mary Sweeney">Mary Sweeney</a>
</td>, <td>Editora y productora
</td>, <td><i>Blue Velvet</i> (1986), <i>Wild at Heart</i> (1990), <i>Twin Peaks</i>, <i>Twin Peaks: Fire Walk with Me</i> (1992), <i>Lost Highway</i> (1997), <i>Mulholland Drive</i> (2001),
</td>, <td>10 producciones
</td>]
Mary Sweeney
Editora y productora
Blue Velvet (1986), Wild at Heart (1990), Twin Peaks, Twin Peaks: Fire Walk

Obtengo la información sobre los colaboadores del cineasta. El rol, las veces que han trabajado juntos y en qué producciones. A continuación creo un dataframe con estos datos.

In [11]:
nombres = []
roles = []
ejemplos = []
producciones = []

for row in tablas.findAll("tr"):
    cells = row.findAll(["th", "td"])
    
    while len(saved_rowspans) < len(cells):
        saved_rowspans.append(None)

    for index, cell in enumerate(cells):
    
        if cell.has_attr("rowspan"):
            rowspan_data = {
                'rows_left': int(cell["rowspan"]),
                'value': cell,
            }
            saved_rowspans[index] = rowspan_data
        else:
            
            if len(cells) != len(saved_rowspans):
                for idx, rowspan_data in enumerate(saved_rowspans):
                    if rowspan_data is not None:
                        
                        inserted_cells.append(rowspan_data['value'])
                        if rowspan_data['rows_left'] == 1:
                            
                            saved_rowspans[idx] = None
                        else:
                            
                            saved_rowspans[idx]['rows_left'] -= 1
    
    cells += inserted_cells
    inserted_cells = []  

    cell_texts = [cell.get_text(strip=True) for cell in cells]
  
    if len(cell_texts) == 4:
        nombres.append(cell_texts[0])
        roles.append(cell_texts[1])
        ejemplos.append(cell_texts[2])
        producciones.append(cell_texts[3])

data = {
    'Nombre': nombres,
    'Rol': roles,
    'Ejemplos': ejemplos,
    'Cantidad de colaboraciones': producciones,
}

colaboradores_df = pd.DataFrame(data)


colaboradores_df

,Nombre,Rol,Ejemplos,Cantidad de colaboraciones
0,Nombre,Rol,Ejemplos,Cantidad de colaboraciones
1,Angelo Badalamenti,Compositor,"Blue Velvet, Twin Peaks, Wild at Heart, Lost H...",12 producciones
2,Mary Sweeney,Editora y productora,"Blue Velvet(1986),Wild at Heart(1990),Twin Pea...",10 producciones
3,Jack Nance,Actor,"Eraserhead, Dune, Blue Velvet, The Cowboy and ...",7 producciones
4,Freddie Jones,"The Elephant Man, Dune, Wild at Heart, Hotel R...",5 producciones,Actor
5,Kyle MacLachlan,"Dune, Blue Velvet, Twin Peaks,Twin Peaks: Fire...",5 producciones,5 producciones
6,Bellina Logan,Actriz,"Wild at Heart, Twin Peaks, On the AireInland E...",4 producciones
7,Grace Zabriskie,Actriz,"Twin Peaks, Wild at Heart, Twin Peaks: Fire Wa...",4 producciones
8,Michael J. Anderson,Actor,"Twin Peaks, Industrial Symphony No. 1, Twin Pe...",Actor
9,Jack Fisk,"Eraserhead,The Straight StoryyMulholland Drive",3 producciones,Actor


En este df se muestra la información de todos los colaboradores pero con informació intercambiada entre las tres últimas columnas.

In [12]:
data_dict = {"Nombre": [], "Rol": [], "Ejemplos": [], "Cantidad de colaboraciones": []}

for row in tablas.findAll("tr"):

    cells = row.findAll(["th", "td"])
  
    if len(cells) == 4:
        data_dict["Nombre"].append(cells[0].get_text(strip=True))
        data_dict["Rol"].append(cells[1].get_text(strip=True))
        data_dict["Ejemplos"].append(cells[2].get_text(strip=True))
        data_dict["Cantidad de colaboraciones"].append(cells[3].get_text(strip=True))

col_dicc_df = pd.DataFrame(data_dict)
col_dicc_df

,Nombre,Rol,Ejemplos,Cantidad de colaboraciones
0,Nombre,Rol,Ejemplos,Cantidad de colaboraciones
1,Angelo Badalamenti,Compositor,"Blue Velvet, Twin Peaks, Wild at Heart, Lost H...",12 producciones
2,Mary Sweeney,Editora y productora,"Blue Velvet(1986),Wild at Heart(1990),Twin Pea...",10 producciones
3,Jack Nance,Actor,"Eraserhead, Dune, Blue Velvet, The Cowboy and ...",7 producciones
4,Bellina Logan,Actriz,"Wild at Heart, Twin Peaks, On the AireInland E...",4 producciones
5,Jeanne Bates,Actriz,EraserheadyMulholland Drive,2 producciones


En este caso, la información en cada columna es correcta, pero solo se muestra para cinco colaboradores. Hago una prueba para los datos que no están en tablas. 

In [13]:
url = "https://es.wikipedia.org/wiki/David_Lynch"

respuesta = requests.get(url)

soup = BeautifulSoup(respuesta.content, 'html.parser')

seccion_filmografia = soup.find('span', {'id': 'Filmograf.C3.ADa'})

for lista in seccion_filmografia.find_all_next('ul'):
    
    nombre_seccion = lista.find_previous('span', {'class': 'mw-headline'}).text
    print(nombre_seccion)
    
    for item in lista.find_all('li'):
        texto = item.text.strip()
        print(texto)
        
    if nombre_seccion.lower() == 'álbumes de estudio':
        break
        
print()

Comerciales para televisión
Obsession by Calvin Klein (1988)
Georgia Coffee (1991)
We Care About New York (1991)
Who is Gio? (1992)
Dangerous Teaser (1993)
Alka-Seltzer Plus (1993)
Tresor (1993)
Barilla Pasta (1993)
Adidas: The Wall (1993)
Revealed (1993)
The Instinct of Life (1993)
Sun Moon Stars (1994)
Dead Leaves, Aunt Droid, Nuclear Winter and Rocket (1997)
Clear Blue Easy (1997)
Parisienne (1998)
Playstation 2: The Third Place (2000)
Nissan Micra (2002)
Álbumes de estudio
BlueBOB (2001)
Dark Night of the Soul (2010)
Crazy Clown Time (2011)
The Big Dream (2013)



In [14]:
url = "https://es.wikipedia.org/wiki/David_Lynch"

respuesta = requests.get(url)

soup = BeautifulSoup(respuesta.content, 'html.parser')

seccion_filmografia = soup.find('span', {'id': 'Filmograf.C3.ADa'})

titulos_obras = []
tipos_obras = []
anios_obras = []

patron_anio = r'\b\d{4}\b'

for lista in seccion_filmografia.find_all_next('ul'):
    nombre_seccion = lista.find_previous('span', {'class': 'mw-headline'}).text
    for item in lista.find_all('li'):
        texto = item.text.strip()
        partes = texto.split(' (')
        titulo_obra = partes[0]
        tipo_obra = nombre_seccion
        anio_obra = None
      
        coincidencias_anio = re.findall(patron_anio, texto)
        if coincidencias_anio:
            anio_obra = coincidencias_anio[0]
      
        titulos_obras.append(titulo_obra)
        tipos_obras.append(tipo_obra)
        anios_obras.append(anio_obra)
    
    if nombre_seccion.lower() == 'álbumes de estudio':
        break

datos = {'Título': titulos_obras, 'Tipo': tipos_obras, 'Año': anios_obras}
lynch_df = pd.DataFrame(datos)

lynch_df = lynch_df[lynch_df['Título'] != '']

pd.set_option('display.max_rows', None)

lynch_df

,Título,Tipo,Año
0,Obsession by Calvin Klein,Comerciales para televisión,1988
1,Georgia Coffee,Comerciales para televisión,1991
2,We Care About New York,Comerciales para televisión,1991
3,Who is Gio?,Comerciales para televisión,1992
4,Dangerous Teaser,Comerciales para televisión,1993
5,Alka-Seltzer Plus,Comerciales para televisión,1993
6,Tresor,Comerciales para televisión,1993
7,Barilla Pasta,Comerciales para televisión,1993
8,Adidas: The Wall,Comerciales para televisión,1993
9,Revealed,Comerciales para televisión,1993


De esta manera se genera el dataframe sin problemas para los comerciales y discografía.

+ **SELENIUM:**

Primero cargamos el código HTML en crudo desde la página de wikipedia.

In [15]:
driver = webdriver.Chrome()

driver.get("https://es.wikipedia.org/wiki/David_Lynch")

print(driver.page_source)

driver.quit()

<html class="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available vector-animations-ready ve-available" lang="es" dir="ltr"><head>
<meta charset="UTF-8">
<title>David Lynch - Wikipedia, la enciclopedia libre</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-f

En este caso, pruebo a extraer las imágenes jpg de la web.

In [16]:
driver = webdriver.Chrome()

driver.get("https://es.wikipedia.org/wiki/David_Lynch")

elementos_imagen = driver.find_elements("tag name", "img")

rutas_imagenes_jpg = []

for elemento_imagen in elementos_imagen:
    ruta_imagen = elemento_imagen.get_attribute("src")
    
    if ruta_imagen.endswith('.jpg'):
        rutas_imagenes_jpg.append(ruta_imagen)

for ruta_imagen in rutas_imagenes_jpg:
    
    respuesta = requests.get(ruta_imagen)
    
    imagen = Image.open(io.BytesIO(respuesta.content))
    
    plt.imshow(imagen)
    plt.axis('off')  
    plt.show()

driver.quit()

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x1036a8a40>

In [17]:
driver = webdriver.Chrome()

driver.get("https://es.wikipedia.org/wiki/David_Lynch")

elementos_imagen = driver.find_elements("tag name", "img")

cantidad_elementos = len(elementos_imagen)


for elemento_imagen in elementos_imagen:
    ruta_imagen = elemento_imagen.get_attribute("src")
    
    
    if ruta_imagen.endswith('.jpg'):
        rutas_imagenes_jpg.append(ruta_imagen)
        
            
for ruta_imagen in rutas_imagenes_jpg:
    
    respuesta = requests.get(ruta_imagen)
    
    print(respuesta)

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [200]>


# EJERCICIO 2
+ Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

In [18]:
ruta_archivo = 'QuotesWord.docx'

documento = Document(ruta_archivo)

for parrafo in documento.paragraphs:
    print(parrafo.text)

QUOTES from TOSCRAPE
‘quotes_df’

CONTEXTO

- Conjunto de citas de personajes famosos de distinta índole y ámbito.
- Por cada cita tenemos la información del autor, la url exacta donde se encuentra y las etiquetas asociadas a esa cita.

CONTENIDO

Dataframe con 100 filas y 9 columnas. 

Columnas:
‘Quotes’: Columna para cada una de las citas.
‘Author’: Indica el autor de la cita.
‘URL’: Representa con un número del 1 al 10 la url en la que está alojada la cita.
‘love’: Columna binaria que marca con 1 o 0 si está etiquetada con ‘love’.
‘Inspirational’: Columna binaria que marca con 1 o 0 si está etiquetada con ‘inspirational’.
‘life’: Columna binaria que marca con 1 o 0 si está etiquetada con ‘life’.
‘humor’: Columna binaria que marca con 1 o 0 si está etiquetada con ‘humor’.
‘books’: Columna binaria que marca con 1 o 0 si está etiquetada con ‘books’.
‘Other_Tags’: Incluye el resto de etiquetas que no están representadas en las anteriores columnas binarias. Las anteriores son las etiquet

In [19]:
ruta_archivo = 'David_Lynch.docx'

documento = Document(ruta_archivo)

for parrafo in documento.paragraphs:
    print(parrafo.text)


QUOTES from DAVID LYNCH’S WIKIPEDIA
‘lynch_df’

CONTEXTO

- Información extraída de la pagina web de wikipedia sobre David Lynch.
- El dataframe muestra la filmografia del director.

CONTENIDO

Dataframe con 68 filas y 3 columnas. 

Columnas:
‘Título’: Columna para el nombre de la obra. 
‘Tipo’: Indica el tipo de obra clasificándolas en 7 categorías: ’Cortometrajes', 	'Mediometrajes', 'Largometrajes','Spots televisivos', 'Series', 	‘Documentales' y 'Álbumes de estudio’. Para el análisis posterior 			podría ser interesante convertir esta columna en una columna 			categórica para cada tipo de obra menos una.
‘Año’: Año de estreno de la obra.

REFERENCIAS:

"https://es.wikipedia.org/wiki/David_Lynch"


# EJERCICIO 3
+ Escoge una web y realiza un web scraping usando la librería Selenium primero y Scrapy después. 

In [20]:
opts = Options()
opts.add_argument("Mozilla/5.0 (Linux; Linux i542 x86_64; en-US) AppleWebKit/603.35 (KHTML, like Gecko) Chrome/55.0.3722.156 Safari/603")

driver = webdriver.Chrome(
    service = Service(ChromeDriverManager().install()), 
    options=opts)

driver.get('https://www.airbnb.es')

propiedades = driver.find_elements(By.XPATH, '//div[contains(@class, "g1qv1ctd")]')

print("Información de las propiedades:")
for propiedad in propiedades:
    nombre = propiedad.find_element(By.XPATH, './/div[@data-testid="listing-card-title"]').text
    precio = propiedad.find_element(By.XPATH, './/div[@data-testid="price-availability-row"]').text
    puntuacion = propiedad.find_element(By.XPATH, '//span[@class="ru0q88m atm_cp_1ts48j8 dir dir-ltr"]').text
   
    print(f"Nombre: {nombre}, Precio: {precio}, Puntuación: {puntuacion}")

driver.quit()

Información de las propiedades:
Nombre: Castelldefels, España, Precio: 304 € 
noche
304 € por noche, Puntuación: 4,9
Nombre: Calafell, España, Precio: 236 € 
noche
236 € por noche, Puntuación: 4,9
Nombre: Sant Pol de Mar, España, Precio: 1.200 € 
noche
1.200 € por noche, Puntuación: 4,9
Nombre: Cubelles, España, Precio: 227 € 
noche
227 € por noche, Puntuación: 4,9
Nombre: Vilanova i la Geltrú, España, Precio: 127 € 
noche
127 € por noche, Puntuación: 4,9
Nombre: Segur de Calafell, España, Precio: 216 € 
noche
216 € por noche, Puntuación: 4,9
Nombre: Mataró, España, Precio: 114 € 
noche
114 € por noche, Puntuación: 4,9
Nombre: Calafell, España, Precio: 177 € 
noche
177 € por noche, Puntuación: 4,9
Nombre: Pineda de Mar, España, Precio: 71 € 
noche
71 € por noche, Puntuación: 4,9
Nombre: Santa Susanna, España, Precio: 123 € 
noche
123 € por noche, Puntuación: 4,9
Nombre: Montgat, España, Precio: 409 € 
noche
409 € por noche, Puntuación: 4,9
Nombre: Villanueva y Geltrú, España, Precio: 1

In [21]:
class AirbnbSpider(scrapy.Spider):
    name = 'airbnb_spider'
    start_urls = ['https://www.airbnb.es']

    def parse(self, response):
        propiedades = response.xpath('//div[contains(@class, "g1qv1ctd")]')

        print("Información de las propiedades:")
        
        for propiedad in propiedades:
            nombre = propiedad.xpath('.//div[@data-testid="listing-card-title"]/text()').get()
            precio = propiedad.xpath('.//div[@data-testid="price-availability-row"]/text()').get()
            puntuacion = propiedad.xpath('//span[@class="ru0q88m atm_cp_1ts48j8 dir dir-ltr"]/text()').get()

            print(f"Nombre: {nombre}, Precio: {precio}, Puntuación: {puntuacion}")

process = CrawlerProcess()
process.crawl(AirbnbSpider)
process.start()


2024-04-30 14:22:10 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2024-04-30 14:22:10 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.10.3, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.10.0, Python 3.11.3 (main, Apr 19 2023, 18:49:55) [Clang 14.0.6 ], pyOpenSSL 23.0.0 (OpenSSL 1.1.1w  11 Sep 2023), cryptography 39.0.1, Platform macOS-13.3.1-arm64-arm-64bit
2024-04-30 14:22:10 [scrapy.crawler] INFO: Overridden settings:
{}
2024-04-30 14:22:10 [py.warnings] WARNING: /Users/angel/anaconda3/lib/python3.11/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUE

Información de las propiedades:
